# Inducing Induction

We present a simple dataset which requires a model to create *induction heads*, defined in [In-context Learning and Induction Heads](https://transformer-circuits.pub/2022/in-context-learning-and-induction-heads/index.html) as:

>a circuit whose function is to look back over the sequence for previous instances of the current token (call it A), find the token that came after it last time (call it B), and then predict that the same completion will occur again (e.g. forming the sequence [A][B] … [A] → [B]).

Our data consist of fixed length sequentially generated strings. The first character is selected randomly from the alphabet. Each subsequent character is generated based on the current character. If the current character occurs previously in the string, where it is followed by X, then the subsequent character is X. Otherwise, the subsequent character is random.

```
....cx...c → x
.........c → ?
```
Here are ten elements of the dataset:
```
izoxhnvffffffffffffffffffffffff
engngngngngngngngngngngngngngng
crlbswrlbswrlbswrlbswrlbswrlbsw
eulyprrrrrrrrrrrrrrrrrrrrrrrrrr
yhxgeeeeeeeeeeeeeeeeeeeeeeeeeee
qgnijuijuijuijuijuijuijuijuijui
yiejfyiejfyiejfyiejfyiejfyiejfy
yikucnsbxrvaqfewsbxrvaqfewsbxrv
omasomasomasomasomasomasomasoma
phokcfdcfdcfdcfdcfdcfdcfdcfdcfd
```

To succeed on this dataset, a network needs to find the previous instance of the current character, and get information about the charter to its right. Because of causal nature of attention (it only looks left), the way this gets implemented in practice is that one attention head copies information to the right, and the next layer will have access to information about two consecutive tokens at once.

Two layers, with one attention head each, are all that is needed in theory. We confirm this in practice, using @karpathy's `makemore` library to train a simple transformer model. We can visualize the resulting attention maps, finding that the "first head copies right, second head looks for previous occurrance" solution is found in practice.

## Generating the dataset

In [1]:
import numpy as np
from collections import defaultdict, OrderedDict
from util import colorize_list

In [2]:
chars = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [3]:
def generate_string(length):
    chain = {}
    string = np.random.choice(chars)
    for i in range(length):
        char = string[-1]
        if char in chain:
            string += chain[char]
        else:
            next_char = np.random.choice(chars)
            string += next_char
            chain[char] = next_char
    return string

In [4]:
def validate_string(string):
    chain = {}
    valid = True
    for i in range(len(string) - 1):
        char = string[i]
        next_char = string[i+1]
        if char in chain and chain[char] != next_char:
            valid = False
        if char not in chain:
            chain[char] = next_char
    return valid

In [5]:
n_lines = 32000

In [6]:
# with open('induction.txt', 'w') as fp:
#     for i in range(n_lines):
#         fp.write(generate_string(30) + '\n')

In [7]:
!head induction.txt

izoxhnvffffffffffffffffffffffff
engngngngngngngngngngngngngngng
crlbswrlbswrlbswrlbswrlbswrlbsw
eulyprrrrrrrrrrrrrrrrrrrrrrrrrr
yhxgeeeeeeeeeeeeeeeeeeeeeeeeeee
qgnijuijuijuijuijuijuijuijuijui
yiejfyiejfyiejfyiejfyiejfyiejfy
yikucnsbxrvaqfewsbxrvaqfewsbxrv
omasomasomasomasomasomasomasoma
phokcfdcfdcfdcfdcfdcfdcfdcfdcfd


## Training the Transformer model

In [8]:
!python makemore.py -i induction.txt -o induction --n-layer 2 --max-steps 2000 --n-head 1

{'input_file': 'induction.txt', 'work_dir': 'induction', 'resume': False, 'sample_only': False, 'num_workers': 4, 'max_steps': 2000, 'device': 'cpu', 'seed': 3407, 'top_k': -1, 'type': 'transformer', 'n_layer': 2, 'n_head': 1, 'n_embd': 64, 'n_embd2': 64, 'batch_size': 32, 'learning_rate': 0.0005, 'weight_decay': 0.01}
number of examples in the dataset: 32000
max word length: 31
number of unique characters in the vocabulary: 26
vocabulary:
abcdefghijklmnopqrstuvwxyz
split up the dataset into 31000 training examples and 1000 test examples
dataset determined that: vocab_size=27, block_size=32
number of parameters: 0.10M
model #params: 105600
step 0 | loss 3.4341 | step time 164.43ms
step 10 | loss 3.3334 | step time 12.54ms
step 20 | loss 3.3051 | step time 12.61ms
step 30 | loss 3.2096 | step time 13.43ms
step 40 | loss 3.1090 | step time 10.76ms
step 50 | loss 2.9756 | step time 10.81ms
step 60 | loss 3.0134 | step time 11.37ms
step 70 | loss 2.9262 | step time 10.98ms
step 80 | loss 2

In [9]:
from makemore import *
from torch.utils.data.dataloader import DataLoader

In [10]:
train_dataset, test_dataset = create_datasets('induction.txt')
batch_loader = DataLoader(train_dataset, 
                        batch_size=32, 
                        num_workers=4, shuffle=False)
vocab_size = train_dataset.get_vocab_size()
block_size = train_dataset.get_output_length()
batch = next(iter(batch_loader))

number of examples in the dataset: 32000
max word length: 31
number of unique characters in the vocabulary: 26
vocabulary:
abcdefghijklmnopqrstuvwxyz
split up the dataset into 31000 training examples and 1000 test examples


In [11]:
config = ModelConfig(vocab_size=vocab_size, block_size=block_size,
                   n_layer=2, n_head=1,
                   n_embd=64, n_embd2=64)
model = Transformer(config)
model.load_state_dict(torch.load(os.path.join('induction', 'model.pt')))

number of parameters: 0.10M


<All keys matched successfully>

In [12]:
# Generate 50 samples
torch.manual_seed(41)
n_samples = 50
X_init = torch.zeros(n_samples, 1, dtype=torch.long).to('cpu')
steps = train_dataset.get_output_length() - 1 # -1 because we already start with <START> token (index 0)
X_samp = generate(model, X_init, steps, do_sample=True).to('cpu')
samples = []
for i in range(X_samp.size(0)):
    # get the i'th row of sampled integers, as python list
    row = X_samp[i, 1:].tolist() # note: we need to crop out the first <START> token
    # token 0 is the <STOP> token, so we crop the output sequence at that point
    word_samp = train_dataset.decode(row)
    samples.append(word_samp)

New data sampled from the transformer model (mostly) matches the generating pattern:

In [13]:
for sample in samples[:10]:
    print(sample)

cgqjmqjmqjmqjmqjmqjmqjmqjmqjmqj
jdrkvahsfvahsfvahsfvahsfvahsfva
bvkduqcwhjzerqcwhjzerqcwhjzerqc
ylpewpewpewpewpewpewpewpewpewpe
pbulmywpbulmywpbulmywpbulmywpbu
qhobruciiiiiiiiiiiiiiiiiiiiiiii
hfilfilfilfilfilfilfilfilfilfil
fatktktktktktktktktktktktktktkt
vlpeacznusnusnusnusnusnusnusnus
uzgoinuzgoinuzgoinuzgoinuzgoinu


In [14]:
invalid_samples = [sample for sample in samples if not validate_string(sample)]
print(f'Out of {len(samples)} samples, {len(invalid_samples)} are invalid:')

for sample in invalid_samples:
    print(sample)

Out of 50 samples, 1 are invalid:
wsgvqyxbpbpbpbpbpbpbpbpbpbpurbp


## Visualize attention layers

In [15]:
attn = [[]]*config.n_layer

def head_hook(layer_idx):
    results = []
    attn[layer_idx] = results
    def get_attn_hook(model, input, output):
        x = input[0]
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k ,v  = model.c_attn(x).split(model.n_embd, dim=2)
        k = k.view(B, T, model.n_head, C // model.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, model.n_head, C // model.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, model.n_head, C // model.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(model.bias[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        results.append(att.detach().numpy())
    return get_attn_hook

In [16]:
for layer_idx in range(config.n_layer):
    model.transformer.h[layer_idx].attn.register_forward_hook(head_hook(layer_idx))

In [17]:
model.eval()
x, y = batch
logits, loss = model(x, y)

In [18]:
def remove_heads(model):
    for layer in model.transformer.h:
        layer.attn._forward_hooks = OrderedDict()
remove_heads(model)

In [19]:
def diag_word(word):
    word_list = []
    for i in range(len(word)):
        word_list.append(word[:i+1] + ' '*(len(word) - i - 1))
    return word_list

In [20]:
head_attn = np.squeeze(np.array(attn))
head_attn.shape # (layer, batch, block, block)

(2, 32, 32, 32)

In [21]:
datapoint_idx = 2

In [22]:
def show_attention(data_index, head_index):
    row = [i.item() for i in batch[0][data_index][1:]]
    word = train_dataset.decode(row)
    colors = head_attn[head_index,datapoint_idx]
    words = diag_word(' ' + word)
    display(colorize_list(words, colors))

The first layer attention head looks one character to the left, copying its information over.

In [23]:
show_attention(2, 0)

The second layer attention head attends to the character *following* the previous instance(s) of the current character (which it can find because of the first head copying information to the right). The value it extracts is that of the character being attended to, which is indeed the next character that should occur in the sequence.

In [24]:
show_attention(2, 1)